# Day 7 : <a href="https://adventofcode.com/2023/day/7">Camel cards</a>

## Part 1

In this problem, we're asked to carry out ordering on a set of cards. The possible cards are (in descending order):

`A`, `K`, `Q`, `J`, `T`, `9`, `8`, `7`, `6`, `5`, `4`, `3`, `2`, `1`

Each set of cards, also called a <i>hand</i> has a corresponding bid. We're asked to find the total winnings of a set of <i>hands</i>. Where each bid corresponding to a hand is multiplied by it's ranking. Where the rankings follow regular poker rules.

As test input data we are given

In [30]:
testData = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".split('\n')

actualData = open("inputs/day7.txt").readlines()

Here the hand `32T3K` has a bid of `765`. If this were to be the worst card of our hand, it would have a value of $1\times 765$. If it were the best of the 5 cards, it would have a value of $5\times 765=3825$. We note the total winnings by the sum of all individual hand winnings. And for our `testData` we have a `testResult` of

In [31]:
testResult = 6440

We begin by importing regular modules

In [32]:
import numpy as np
import backend
import re

There are different kinds of hands. First, we write a function that classifies these hands. We do so by assigning numerical values to each type of hand, which we can sort by later.

In [33]:
high_card = 1
one_pair = 2
two_pair = 3
three_of_a_kind = 4
full_house = 5
four_of_a_kind = 6
five_of_a_kind = 7

Let us first define a function that returns a dictionary with all cards and their relative number of appearances.

In [34]:
def cardsInHand(hand : str) -> dict:
    cards = np.array([card for card in hand], dtype = str) # Create a numpy array
    cardsDict = dict() # Create an empty dictionary first
    for card in cards : cardsDict.update({card : np.sum(cards == card)})
    return cardsDict

cardsInHand("AA36J")

{'A': 2, '3': 1, '6': 1, 'J': 1}

The next task is to create a mapping of hands to numbers, so we can simply sort the numbers. A first note is that there are 7 types of hand strengths. We could therefore first assign a score of 1 to 7 to each hand type. Luckily, we have already done so in one of the code sections above. The only thing left to do is to create a function that maps a hand to this score.

In [35]:
def handStrength(hand : str) -> int:
    assert len(hand) == 5, "A hand should have five cards."
    cardFrequencies = np.array(list(cardsInHand(hand).values())) # For the strenght of a hand we are only interested in the frequencies of cards
    if np.any(cardFrequencies == 5) : return five_of_a_kind
    if np.any(cardFrequencies == 4) : return four_of_a_kind
    if np.any(cardFrequencies == 3) and np.any(cardFrequencies == 2) : return full_house
    if np.any(cardFrequencies == 3) and not np.any(cardFrequencies == 2) : return three_of_a_kind
    if np.sum(cardFrequencies == 2) == 2 : return two_pair
    if np.sum(cardFrequencies == 2) == 1 : return one_pair
    return high_card

handStrength("AADDE")

3

The problem now is that we do not take into account the quality of the first to last card. Consider the example below:

In [36]:
handStrength("AAAJJ"), handStrength("22288")

(5, 5)

Clearly, we should prefer the hand `AAAJJ` over `22288`. First, let us define the ordering of single cards. That is, we assign values to each card, where a higher number corresponds to a higher value. We then create a function called `cardValue` that returns the card-value of a card.

In [37]:
cards = np.array(["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"])
cardValues = np.flip(np.arange(len(cards)))

def cardValue(card : str) -> int:
    return (cardValues[cards == card]/(2*cardValues.max()))[0]

cardValue("2")

0.0

Now, we should create some form of ordering. In this case, the first card gets compared first. First we need to decide the value of each card in a hand and take some form of sum. I will call this the 'additional hand value' which is the hand strength, plus some number between 0 and 1 denoting the value of a hand. Therefore we know that if two hands have differing strengths, the stronger card will always have a higher ranking than the the hand with the lower strength. The rank of a hand is then simply the strength of a hand plus its additional hand value.

To come up with the hand value, we need to note that the first card counts the most towards the additional hand value. For this, cardValue needs to be mapped on a scale between 0 and 1. Which is done in the function `cardValue`. To compute the additional hand value we take 1 times the first card value + 0.1 times the second card value + 0.01 times the third card value etc. This guarantees that we create a correct ordering, based on the assumption that the first card counts the most.

In [38]:
def additionalHandValue(hand : str) -> float:
    additionalValue = 0.0
    for card, factor in zip(hand, 1/(10**np.arange(len(hand)))):
        additionalValue += factor * cardValue(card)
    return(additionalValue)

def handRank(hand : str) -> int:
    handRank = handStrength(hand) + additionalHandValue(hand)
    return handRank


Now we can once again compare the two hands we've shown before:

In [39]:
handRank("AAAJJ"), handRank("22288")

(5.5554125, 5.000275)

In this case, we see that both hands are full houses and have the same strength, but the first card has a higher additional value, so it has a higher rank. We can now sort all our hands by rank! As we want to keep track of both the hands and their bids, we create an indirect sorting function (`argSortHands`), which just returns the indexes of the elements in `hands` in sorted order.

In [40]:
def argSortHands(hands : list):
    return np.argsort([handRank(hand) for hand in hands])

argSortHands(["AAAJJ", "22288"])

array([1, 0], dtype=int64)

Great! Now we can sort both our bids corresponding to the hands!

In [44]:
def day7_part1(input : str) -> int:
    hands = np.array([], dtype = str)
    bids = np.array([])
    for line in input:
        matches = re.match("(.....) (.+)", line)
        hand, bid = matches.group(1), int(matches.group(2))
        hands = np.append(hands, hand)
        bids = np.append(bids, bid)
    
    sortedIndexes = argSortHands(hands)
    sortedHands = hands[sortedIndexes]
    sortedBids = bids[sortedIndexes]
    print(sortedHands)

    return int(np.sum(sortedBids * np.arange(1, len(sortedBids)+1)))

As shown above we finally multiply all bids by 1, 2, 3, etc, depending on their rank. Now let's run our test case!

In [42]:
backend.test(day7_part1, 6440, input = testData)

Completed in  0.00 seconds.
Answer: 
6440
Test succeeded.


Great! Finally, let's run it on our actual input data.

In [45]:
backend.run(day7_part1, input = actualData)

['23567' '249TK' '24AJ3' '254JK' '26TK5' '26A87' '27638' '285K3' '2TAQK'
 '2J54A' '2J6K9' '325J9' '32J6T' '2K43Q' '2QA87' '2A38K' '34628' '346KQ'
 '347JA' '34QK8' '35Q7T' '364K2' '36T8K' '374K5' '37KAJ' '389A5' '38T79'
 '39287' '3TQ7J' '3JK42' '4276T' '3K284' '3K549' '3KA2Q' '3A48Q' '459QA'
 '47T2Q' '48T9Q' '492QJ' '4J93T' '4J9TQ' '4Q269' '4Q2T5' '4JK37' '5249Q'
 '4JAT7' '4QT2K' '4QK82' '53782' '4A37Q' '53AK9' '54TQK' '57A9T' '57AJ3'
 '5892A' '58TK3' '5928Q' '58K74' '5986A' '5T264' '59K37' '5TJ83' '5JK2A'
 '6297T' '62A47' '63479' '5K829' '6382Q' '63T89' '63QJ5' '5KAQ3' '6479T'
 '64QTJ' '65JA9' '6729J' '67324' '67A9Q' '69243' '6932T' '68QA7' '6Q2TA'
 '723A4' '72689' '72J58' '6K2A8' '72A6T' '6A285' '6KA74' '6A4JT' '6A987'
 '74T36' '75A42' '76J5A' '76K52' '7935J' '7T846' '7J235' '7J689' '7J9QK'
 '7JT56' '7JTQ8' '7Q5A3' '8273K' '7QJTA' '7K42Q' '7K463' '7K63A' '7K85J'
 '83TK9' '7A396' '8465J' '7AKJ9' '879Q5' '89J45' '8T36A' '8T4K2' '8JT2K'
 '8Q2K9' '8Q9JA' '8KT74' '8A5K4' '94T85' '94K3J' '9

251142035